In [1]:
from BaseEncoder import BaseEncoder
from transformers import AutoTokenizer, TFAutoModel

/home/rifoag/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
sentence = 'Aku seorang kapiten'

In [3]:
base_encoder = BaseEncoder()

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [9]:
base_encoder.encode(sentence)

TypeError: encode() missing 1 required positional argument: 'sentence'